In [4]:
from ipycanvas import RoughCanvas
canvas = RoughCanvas()

canvas.stroke_rect(100, 100, 100, 100)
canvas.fill_rect(50, 50, 100, 100)

canvas.stroke_circle(300, 300, 100)
canvas.fill_circle(350, 350, 100)

canvas.stroke_line(200, 200, 300, 300)

canvas.stroke_line(0, 0, canvas.width, canvas.height)

canvas


RoughCanvas()

In [51]:
import numpy as np
from ipywidgets import interact
from ipycanvas import hold_canvas, RoughCanvas, Canvas

canvas = Canvas()

width, height = 320, 200

n = 10
x, y = np.meshgrid(np.linspace(0, width, n), np.linspace(0, height, n))
w = np.ones(x.shape)
p1s = np.stack([x, y, w], axis=-1).reshape((-1, 3))

# external camera
def camera(f, ay):
    R = np.array([
        [np.cos(ay), 0, np.sin(ay)],
        [0, 1, 0],
        [-np.sin(ay), 0, np.cos(ay)],
    ])
    t = np.array([-width/2, -height/2, -500])
    K = np.array([
        [f,  0, canvas.width/2],
        [0, f, canvas.height/2],
        [0, 0, 1],
    ])
    return np.dot(K, np.hstack([R, t[:, None]]))


def homogenize(euclidian):
    return np.vstack((euclidian, np.ones((1, euclidian.shape[1]))))


def dehomogenize(homogenous):
    return homogenous[:-1, :] / homogenous[-1, :]


def stroke_line(canvas, p1, p2):
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    canvas.stroke_line(x1, y1, x2, y2)

    
def stroke_lines(canvas, P, lines):
    # split into from and to coordinates per line
    p1s, p2s = np.split(lines, 2, axis=1)
    p1s = np.squeeze(p1s, axis=1)
    p2s = np.squeeze(p2s, axis=1)
    # project with camera
    pp1s = dehomogenize(np.dot(P, homogenize(p1s.T))).T
    pp2s = dehomogenize(np.dot(P, homogenize(p2s.T))).T

    with hold_canvas(canvas):
        for p1, p2 in zip(pp1s, pp2s):
            stroke_line(canvas, p1, p2)
        
def rectangle(width, height):
    """Create lines for a rectangle"""
    return np.array([
        [[0, 0, 0], [width, 0, 0]],
        [[width, 0, 0], [width, height, 0]],
        [[width, height, 0], [0, height, 0]],
        [[0, height, 0], [0, 0, 0]],
    ])
        
def update_focal(f, ay):
    K = np.array([
        [f,  0, width/2],
        [0, f, height/2],
        [0, 0, 1],
    ])
    #normals = np.dot(np.linalg.inv(K), p1s.T).T
    #p2s = p1s + normals
    #lines = np.stack([p1s, p2s], axis=1)  # same shape as matplotlib
    lines = rectangle(width, height)

    canvas.clear()
    P = camera(10, ay)
    stroke_lines(canvas, P, lines)
    

interact(update_focal, f=(0.1, 20), ay=(0, 2*np.pi))
canvas


interactive(children=(FloatSlider(value=10.049999999999999, description='f', max=20.0, min=0.1), FloatSlider(v…

Canvas()